# Question 5

In [21]:
import numpy as np
import pandas as pd
import nltk
import torch
nltk.download('omw-1.4')
import checklist
from copy import deepcopy
from checklist.editor import Editor
from checklist.perturb import Perturb
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report, confusion_matrix
np.random.seed(42)

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/nedim.azar/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [4]:
# Loading the subset data
df = pd.read_csv('data/olid-subset-diagnostic-tests.csv')

# Creating a version with typos
df_typos = deepcopy(df)
df_typos['text'] = df['text'].apply(Perturb.add_typos)

df.to_csv('data/no_typos.csv')
df_typos.to_csv('data/typos.csv')

### We got the predictions on Google colab because it wouldn't work locally due to GPU and CUDA issues. We load those predictions in...

In [33]:
predictions = pd.read_csv('predictions/No_Typos_Prediction.csv').drop(['Unnamed: 0'], axis=1)

predictions_typos = pd.read_csv('predictions/Typos_Prediction.csv').drop(['Unnamed: 0'], axis=1)

y_true = predictions['labels']
y_pred = predictions['Predictions']
y_pred_typos = predictions_typos['Predictions']
pos_label = 1

## Precision Scores

In [26]:
# No Typos
precision_score(y_true=y_true, y_pred=y_pred, pos_label=pos_label)

0.9459459459459459

In [27]:
# Typos
precision_score(y_true=y_true, y_pred=y_pred_typos, pos_label=pos_label)

0.9459459459459459

In [29]:
list(y_pred) == list(y_pred_typos)

True

In [35]:
y_pred

0     1
1     0
2     1
3     0
4     1
     ..
95    0
96    0
97    0
98    0
99    0
Name: Predictions, Length: 100, dtype: int64

In [34]:
y_pred_typos

0     1
1     0
2     1
3     0
4     1
     ..
95    0
96    0
97    0
98    0
99    0
Name: Predictions, Length: 100, dtype: int64

In [37]:
predictions_typos

,id,text,labels,Predictions
0,89200,@USER @USER Who the hell does he think hei s?,1,1
1,71294,#BREAKING. #Greece: Molotov cocktails fly afte...,1,0
2,55633,"#OrrinHatch I can’t believe this sexist ,clue...",1,1
3,16856,@USER @USER I'll use that oen the next time im...,1,0
4,26072,0-1 lots my acca on the first fucking fight cba,1,1
...,...,...,...,...
95,45518,U@SER He is obviously getting suspended. He is...,0,0
96,51610,#Canada - EXCLUSIV:E #Trudeau #Liberals leave ...,0,0
97,26758,@SUER @USER ...than why did you show us how ho...,0,0
98,30718,@USER @USER @USER You have yet to answer what ...,0,0
